In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import platform
import pathlib

In [ ]:
## Paths Input Here
analysis = '1_way_anova/lobe_damage_scores_csf'
if platform.uname().system == 'Darwin': #------------------------------Mac OS X---------------------------------------------------------------
    conn_path = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STN_WURZBURG/damage_scores/all_patient_damage_scores_csf.csv'
    clin_path = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/patient_data/AD_Clinical_Data_CDR_ADAS_COG_13.xlsx'
    # clin_path = 'path to clinical values'
    out_dir = os.path.join(os.path.dirname(conn_path), f'{analysis}')
    print('I will save to:', out_dir)
    x_roi_names = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/analyses/roi-roi_correl/matrix_corrMx_names.csv'
    #roi_names = '<path to roi name location>'
    print('I have set pathnames in the Mac style')
else: #----------------------------------------------------------------Windows----------------------------------------------------------------
    conn_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_AvgR.csv'
    clin_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\patient_data\AD_Clinical_Data_CDR_ADAS_COG_13.xlsx'
    # clin_path = 'path to clinical values'
    out_dir = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\stats'
    #out_dir = r'path to out dir here'
    x_roi_names = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_names.csv'
    #roi_names = '<path to roi name location>'
    print('I have set pathnames in the Windows style')
    
## Do you want to save the reults?
save = True
if save:
    if os.path.exists(out_dir) != True:
        os.makedirs(out_dir)

## Import Dataframe

In [ ]:
data_df = pd.read_csv(conn_path)

#Prepare the dataframe for statsmodels
data_df = data_df.reset_index(drop=True)
# data_df.dropna(inplace=True)
try:
    data_df.pop('Unnamed: 0')
    data_df.pop('% Change from baseline (CDR)')
except:
    print('none to pop')
data_df = data_df.rename(columns={'% Change from baseline (ADAS-Cog11)': 'percent_change_adascog11'})#, '07_default': 'default', '02_somatomotor_seed': 'somatomotor', '03_dorsal_attention': 'dorsal', '01_visual_seed': 'visual', '04_ventral_attention': 'ventral', '05_limbic': 'limbic', '06_frontoparietal': 'frontoparietal'})
display(data_df)

In [ ]:
# Clean Dataframe
index_of_first_column_to_include = 0
group_variable = 'Lobe'
independent_variable = 'Damage Score'
#----------------------------------------------------------------DO NOT TOUCH----------------------------------------------------------------
try:
    data_df = data_df.copy().iloc[:, index_of_first_column_to_include:]
    names = data_df.columns.to_list()
    data_df.columns = [name.split('_')[1].replace('_', ' ').capitalize() for name in names]
    data_df
except:
    pass

In [ ]:
#USE IF YOU MUST UNMELT THE DATAFRAME
# column = None
# values = None
# #----------------------------------------------------------------DO NOT TOUCH
# df_unmelted = data_df.pivot(index=None, columns=column, values=values)
# data_df = df_unmelted
# display(df_unmelted)

## Massage Dataframe for ANOVA

## ANOVA Test Time

In [ ]:
index_of_first_column_to_include = 0
#----------------------------------------------------------------DO NOT TOUCH
#Generate string to enter into ANOVA
def remove_apostrophes(input_string):
    return input_string.replace("'", "")

columns_string = data_df.columns.values.tolist()[index_of_first_column_to_include:]
data_df_string = []
for i in range(0, len(columns_string)):
    name = f'data_df["{columns_string[i]}"]'
    data_df_string.append(name)
    
encapsulated_string = str(data_df_string)
encapsulated_string = encapsulated_string.replace("'", "")
print('Here is a list to set up the ANOVA')
print(encapsulated_string)

In [ ]:
#Begin the ANOVA
from scipy.stats import f_oneway
# box_df = melted_df.copy()
# names = data_df.columns.values.tolist()[1:]

##----------------------------------------------------------------COPY THE STRING PRINTED ABOVE INTO HERE----------------------------------------------------------------
fvalue, pvalue = f_oneway(
data_df["Temporal"], data_df["Frontal"], data_df["Parietal"], data_df["Cerebellum"], data_df["Insula"], data_df["Occipital"]
)
##----------------------------------------------------------------END COPYIN--------------------------------------------------------
    #box_df['mem'], box_df['vis'], box_df['som'], box_df['dor'], box_df['ven'], 
                        #   box_df['lim'], box_df['fro'], box_df['def'])

results_df = pd.DataFrame({'ANOVA f-value': fvalue, 'ANOVA p-value': pvalue}, index=['stat'])
display(results_df)

In [ ]:
# melted_df.to_csv(os.path.join(out_dir, ('descrptive_stats_of_inputs.csv')))
results_df.to_csv(os.path.join(out_dir, 'anova_results.csv'))
print(f'saved to: \n {out_dir}')

## Perform Post Hoc Testing

In [ ]:

#----------------------------------------------------------------DO NOT TOUCH----------------------------------------------------------------
# # Prep Dataframe
df_to_melt = data_df.copy().transpose()
df_to_melt[group_variable] = df_to_melt.index.to_list()
df_to_melt
melted_df = pd.melt(df_to_melt, id_vars=[group_variable], value_vars=df_to_melt.columns.values.tolist()[0:-1], value_name=independent_variable)
display(melted_df)

In [ ]:
#Perform posthoc
from scikit_posthocs import posthoc_tukey

tukey_df = posthoc_tukey(melted_df, val_col=independent_variable, group_col=group_variable)
display(tukey_df)

In [ ]:
tukey_sig_is_one_df = tukey_df.copy()
tukey_sig_is_one_df.where(tukey_sig_is_one_df > 0.05, 1)
# tukey_sig_is_one_df[(tukey_sig_is_one_df > 0.05)] = 0
# tukey_sig_is_one_df[(tukey_sig_is_one_df < 0.05)] = 1
# sns.clustermap(tukey_sig_is_one_df, cmap='rocket_r')
sigplot = plt.figure()
sns.heatmap(tukey_sig_is_one_df, cmap='binary', linewidth=2.0, linecolor='k')

In [ ]:
#Save the Tukey Information
if save:
    if os.path.isdir(out_dir) != True:
        os.mkdir(out_dir)
    tukey_df.to_csv(os.path.join(out_dir, ('posthoc_tukey_dataframe.csv')))
    sigplot.savefig(os.path.join(out_dir, ('posthoc_tukey_significance_plot.png')))
    print(f'saved to: \n {out_dir}')

In [ ]:
#prepare to plot hte post-hoc analysis
#remove redundant component of matrix and drop NANs, then melt the DF.
from natsort import natsort_keygen, order_by_index, index_natsorted
remove = np.tril(np.ones(tukey_df.shape), k=0).astype('bool')
tukey_df[remove] = np.nan
molten_df = tukey_df.melt(ignore_index=False).reset_index().dropna()

#----------------------------------------------------------------Organize the Datafram--------------------------------
# create a new column that contains the position of each value within its group
# melted_df['position'] = melted_df.groupby('id').cumcount()

# # sort the dataframe by the group column and the new position column
# melted_df = melted_df.sort_values(['id', 'position'])

# # drop the position column
# melted_df = melted_df.drop(columns=['position'])

# molten_df = molten_df.reindex(index=order_by_index(molten_df.index, index_natsorted(molten_df.variable)))
molten_df

In [ ]:
from statannotations.Annotator import Annotator
from natsort import natsort
#----------------------------------------------------------------Organize the Figure
# define the value you want to move the rows afte
order = natsort.natsorted(np.unique(melted_df[group_variable]))
print(order)

#----------------------------------------------------------------Identify Values and Annotate
#Extract information from the melted dataframe above
pairs = [(i[1]["index"], i[1]["variable"]) for i in molten_df.iterrows()]
p_values = [i[1]["value"] for i in molten_df.iterrows()]

# Identify only nonsignificant pairs
only_nonsig = False
if only_nonsig:
    ns_vals = []
    ns_pairs = []
    for i in range(0, len(p_values)):
        if p_values[i] > 0.05:
            ns_vals.append(p_values[i])
            ns_pairs.append(pairs[i])
    pairs = ns_pairs
    p_values = ns_vals

#Identify only significant pairs
only_sig = True
if only_sig:
    s_vals = []
    s_pairs = []
    for i in range(0, len(p_values)):
        if p_values[i] < 0.05:
            s_vals.append(p_values[i])
            s_pairs.append(pairs[i])
    pairs = s_pairs
    p_values = s_vals

#----------------------------------------------------------------Plot
#Create the plot
sns.set_style('white')
fig = plt.figure(figsize=(15,10))
variable = np.unique(melted_df[group_variable])
ax = sns.boxplot(data=melted_df, x=group_variable, y=independent_variable, order=order, color='grey')

annotator = Annotator(ax, pairs, data=melted_df, x=group_variable, y=independent_variable, order=order)

annotator.configure(text_format="star", loc="inside")
annotator.set_pvalues_and_annotate(p_values)

ax.set_xlabel(group_variable)
ax.set_ylabel(independent_variable)

In [ ]:
# Save outcomes
if save:
    if os.path.isdir(out_dir) != True:
        os.mkdir(out_dir)
    fig.savefig(os.path.join(out_dir, ('annotated_boxplot.png')))
    print(f'saved to: \n {out_dir}')

In [ ]:
print(melted_df.iloc[50:200, :])